<a href="https://colab.research.google.com/github/YungSpinach/org_llm/blob/main/NEF_LLM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit PyPDF2 langchain sentence-transformers faiss-cpu transformers python-docx torch pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("YOUR_AUTH_TOKEN")


In [ ]:
app_code = """
import os
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
from docx import Document
import torch

# Load documents
def load_documents(folder_path):
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith(".pdf"):
            reader = PdfReader(file_path)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
            documents.append(text)
        elif filename.endswith(".txt"):
            with open(file_path, "r", encoding="utf-8") as file:
                text = file.read()
            documents.append(text)
        elif filename.endswith(".docx"):
            doc = Document(file_path)
            text = "\\n".join([para.text for para in doc.paragraphs])
            documents.append(text)
    return documents

# Process documents
def process_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = embedding_model.encode([chunk.page_content for chunk in chunks])
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings))
    return chunks, index, embedding_model

# Load LLM
def load_llm(model_name="deepseek-ai/deepseek-llm"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    return tokenizer, model

# Semantic search
def search(query, index, chunks, embedding_model, top_k=3):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    return [chunks[i].page_content for i in indices[0]]

# Generate response
def generate_response(query, context, tokenizer, model):
    input_text = f"Context: {context}\\n\\nQuestion: {query}\\n\\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
    outputs = model.generate(**inputs, max_length=500)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Streamlit App
st.title("LLM Document Querying App")

# Set your document folder path (for uploaded files)
folder_path = "/content/documents/"
os.makedirs(folder_path, exist_ok=True)

# Load and process documents
documents = load_documents(folder_path)
if not documents:
    st.error("No documents found. Please upload files first.")
else:
    chunks, index, embedding_model = process_documents(documents)
    tokenizer, model = load_llm()

# Query interface
query = st.text_input("Enter your query:")
if query:
    relevant_chunks = search(query, index, chunks, embedding_model)
    context = "\\n".join(relevant_chunks)
    response = generate_response(query, context, tokenizer, model)
    st.write("Response:", response)
"""

# Save as `app.py`
with open("app.py", "w") as f:
    f.write(app_code)

print("✅ `app.py` has been created successfully!")


In [ ]:
!ls /content/


In [ ]:
from google.colab import files
uploaded = files.upload()  # Select files from your computer


In [ ]:
# Run Streamlit in the background
!streamlit run app.py &

# Expose port 8501 using ngrok
from pyngrok import ngrok

public_url = ngrok.connect(port=8501)
print("🌍 Public URL:", public_url)


After running Step 8, you will see an output like:

 Public URL: http://randomstring.ngrok.io
Click the link to open your Streamlit app.
Upload your documents and start querying.
